In [32]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import os
import shutil

In [5]:
import re

# Regular expression to match the time format 00:00:00.000
time_pattern = re.compile(r'\b\d{2}:\d{2}:\d{2}\.\d{3}\b')

def remove_time_format(file_path, output_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Open a new file to write the filtered data
    with open(output_path, 'w') as output_file:
        for line in lines:
            # Remove all occurrences of the time format from the line
            cleaned_line = time_pattern.sub('', line)
            output_file.write(cleaned_line)

# Example usage
input_file = 'input.txt'
output_file = 'patients.txt'
remove_time_format(input_file, output_file)

print(f"Data with time format removed has been written to {output_file}")


Data with time format removed has been written to patients.txt


In [11]:
patients_df = pd.read_csv(r"E:\medical-records-model\data 100\people.csv")
admissions_df = pd.read_csv(r"E:\medical-records-model\data 100\Admissions_people.csv")
diagnoses_df = pd.read_csv(r"E:\medical-records-model\data 100\diagnosis.csv")
labs_df = pd.read_csv(r"E:\medical-records-model\data 100\lab_results.csv")

In [12]:
patients_df.head()

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty
0,FB2ABB23-C9D0-4D09-8464-49BF0B982F0F,Male,28-12-1947,Unknown,Married,Icelandic,18.08
1,64182B95-EB72-4E2B-BE77-8050B71498CE,Male,18-01-1952,African American,Separated,English,13.03
2,DB22A4D9-7E4D-485C-916A-9CD1386507FB,Female,25-07-1970,Asian,Married,English,6.67
3,6E70D84D-C75F-477C-BC37-9177C3698C66,Male,04-01-1979,White,Married,English,16.09
4,C8556CC0-32FC-4CA5-A8CD-9CCF38816167,Female,11-04-1921,White,Married,English,18.20


In [13]:
admissions_df.head()    

,PatientID,AdmissionID,AdmissionStartDate,AdmissionEndDate
0,7A025E77-7832-4F53-B9A7-09A3F98AC17E,7,12-10-2011,22-10-2011
1,DCE5AEB8-6DB9-4106-8AE4-02CCC5C23741,1,11-02-1993,24-02-1993
2,DCE5AEB8-6DB9-4106-8AE4-02CCC5C23741,2,28-11-2002,04-12-2002
3,DCE5AEB8-6DB9-4106-8AE4-02CCC5C23741,3,19-07-2011,25-07-2011
4,886B5885-1EE2-49F3-98D5-A2F02EB8A9D4,1,03-12-1994,20-12-1994


In [14]:
labs_df.head()

,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime
0,1A8791E3-A61C-455A-8DEE-763EB90C9B2C,1,URINALYSIS: RED BLOOD CELLS,1.8,rbc/hpf,01-07-1992
1,1A8791E3-A61C-455A-8DEE-763EB90C9B2C,1,METABOLIC: GLUCOSE,103.3,mg/dL,30-06-1992
2,1A8791E3-A61C-455A-8DEE-763EB90C9B2C,1,CBC: MCH,35.8,pg,30-06-1992
3,1A8791E3-A61C-455A-8DEE-763EB90C9B2C,1,METABOLIC: CALCIUM,8.9,mg/dL,30-06-1992
4,1A8791E3-A61C-455A-8DEE-763EB90C9B2C,1,CBC: RED BLOOD CELL COUNT,4.8,m/cumm,01-07-1992


In [15]:
diagnoses_df.head()

,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,80AC01B2-BD55-4BE0-A59A-4024104CF4E9,2,M01.X,Direct infection of joint in infectious and pa...
1,80AC01B2-BD55-4BE0-A59A-4024104CF4E9,3,D65,Disseminated intravascular coagulation [defibr...
2,80AC01B2-BD55-4BE0-A59A-4024104CF4E9,4,C92.1,"Chronic myeloid leukemia, BCR/ABL-positive"
3,80AC01B2-BD55-4BE0-A59A-4024104CF4E9,5,M05.51,Rheumatoid polyneuropathy with rheumatoid arth...
4,6A57AC0C-57F3-4C19-98A1-51135EFBC4FF,1,C91.00,Acute lymphoblastic leukemia not having achiev...


In [21]:
# Pivot the labs dataframe so that each lab type becomes a column with appropriate aggregation
labs_pivot_df = labs_df.pivot_table(
    index=["PatientID", "AdmissionID"],
    columns="LabName",
    aggfunc={
        'LabValue': 'mean',   # Aggregate numerical lab values by mean
        'LabUnits': 'first',  # Take the first available unit for each lab test
        'LabDateTime': 'first'  # Take the first available date for each lab test
    }
)

# Flatten the multi-index columns
labs_pivot_df.columns = ['_'.join(col).strip() for col in labs_pivot_df.columns.values]

# Drop columns related to LabDateTime
labs_pivot_df = labs_pivot_df[[col for col in labs_pivot_df.columns if 'LabDateTime' not in col]]

# Merge the pivoted labs data with the rest of the data
final_df = pd.merge(
    patient_admissions_diagnoses_df,
    labs_pivot_df,
    on=["PatientID", "AdmissionID"],
    how="left",
)


In [22]:
final.head()

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty,AdmissionID,AdmissionStartDate,AdmissionEndDate,...,LabValue_METABOLIC: CHLORIDE,LabValue_METABOLIC: CREATININE,LabValue_METABOLIC: GLUCOSE,LabValue_METABOLIC: POTASSIUM,LabValue_METABOLIC: SODIUM,LabValue_METABOLIC: TOTAL PROTEIN,LabValue_URINALYSIS: PH,LabValue_URINALYSIS: RED BLOOD CELLS,LabValue_URINALYSIS: SPECIFIC GRAVITY,LabValue_URINALYSIS: WHITE BLOOD CELLS
0,FB2ABB23-C9D0-4D09-8464-49BF0B982F0F,Male,28-12-1947,Unknown,Married,Icelandic,18.08,1,07-10-1968,10-10-1968,...,107.200000,0.833333,89.175000,4.866667,140.650000,8.066667,6.133333,1.9250,1.0,3.366667
1,FB2ABB23-C9D0-4D09-8464-49BF0B982F0F,Male,28-12-1947,Unknown,Married,Icelandic,18.08,2,11-06-1974,13-06-1974,...,103.566667,1.050000,126.200000,4.600000,135.766667,8.600000,5.333333,0.7500,1.0,3.350000
2,FB2ABB23-C9D0-4D09-8464-49BF0B982F0F,Male,28-12-1947,Unknown,Married,Icelandic,18.08,3,09-11-2009,26-11-2009,...,104.433333,0.821429,95.244444,4.975000,142.607692,8.046154,5.700000,1.8750,1.0,3.615385
3,64182B95-EB72-4E2B-BE77-8050B71498CE,Male,18-01-1952,African American,Separated,English,13.03,1,01-08-1976,11-08-1976,...,106.283333,0.857143,99.985714,4.414286,136.900000,6.485714,5.600000,1.8625,1.0,3.712500
4,64182B95-EB72-4E2B-BE77-8050B71498CE,Male,18-01-1952,African American,Separated,English,13.03,2,04-06-1991,08-06-1991,...,113.200000,0.500000,87.875000,3.700000,133.900000,8.150000,6.333333,1.3250,1.0,3.033333


In [27]:
final.to_csv("final.csv")

In [29]:
final = pd.read_csv("final.csv")

In [30]:
# Replace any missing values (NaN) with 0 in the final dataframe
final = final.fillna(0)

In [ ]:
final.head()